# Brute Force Ising Model


### Statistical mechanics
Using *statistical mechanics*, the probability of finding a given state is given by
$$ P = \frac{e^{-\beta H}}{Z} $$
Where $\beta = \frac{1}{k_bT}$ and $Z$ is the **partition function**, given by 
$$ Z = \sum_{states}e^{-\beta H} $$
However, this requires calculating energies of *all of the given states*, which scales as $2^{n_{spins}}$. 

This is pretty inefficient (i.e. for a 16x16 grid, $2^{16*16}\approx n_{\text{atoms in universe}}$)

### Approximating the partition function
A more reasonable approach is to randomly sample states and approximate an *average* energy of all states $\bar{E}$, which is then used to calculate $Z$ 

$$ Z = n_{states}*\bar{E}$$

However, even this is flawed as there are more low-energy states than high-energy ones. For example, consider a randomly shuffled deck of cards. If 'disorder' is lower energy, there's many more 'disordered' configurations of the deck than 'ordered.' This makes it harder to calculate a true average


### import required libraries

In [1]:
import numpy as np
from numpy import random
from numba import jit

### create functions for generating arrays, calculating energy and magnetism

In [2]:
#Function to create random square array
def gen_random_conf(n):
    #input: n = length of array = width of array
    #output: random 10x10 array (-1/2's and +1/2's)
    
    A = np.random.choice([-1,1], size=(n,n))
    
    return(A)

In [3]:
@jit(nopython=True)
def calculate_energy(spin):
    
    #Determine if array is square
    spin_shape = spin.shape
    
    #Should end if array is not square
    if spin_shape[0] != spin_shape[1]:
        print("This is not a square matrix")
    #Continues code if array is square    
    elif spin_shape[0] == spin_shape[1]:
        N = spin_shape[0]
        
        energy = 0
        J = 1

        for i in range(N):  # 0, N-1
          for j in range(N): # 0, N-1
            # calculate four interactions
            ii = i + -1
            if (ii < 0): 
                ii = ii + N
            if (ii >=N): 
                ii = ii - N
            energy = energy + spin[i,j]*spin[ii, j] #spin--> array name

            
            jj = j + -1
            if (jj < 0): 
                jj = jj + N
            if (jj >=N): 
                jj = jj - N
            energy = energy + spin[i,j]*spin[i, jj] 

            energy = -J * energy * 0.5
    
    return energy

In [4]:
#Sums all indices of array(magnetic sum)
def magnetism_sum(spin):
    magnetism = np.sum(spin)
    return magnetism

### brute force loop
- plot energy and magnetism vs. temperature
- I don't have access to this code, Ava may have the only copy